In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
import json

In [4]:
session=requests.session()

In [5]:
RETRY_COUNT = 1 

In [6]:
class PartnershipFirmList:
    def __init__(self,icai_number):
        self.icai_number=icai_number
        self.session=None
        self.user_agent=None
        self.url_link="https://eicmai.in/external/Transaction/OfficesAndFirm/PartnershipFirmList.aspx"
        
    def home_page(self):
        url_link="https://eicmai.in/external/Transaction/OfficesAndFirm/PartnershipFirmList.aspx"
        response=session.get(self.url_link)
        if response.status_code == 200:
            return response
        else:
            return "No response"
        
    def parse_with_lxml(self,home_page_response):
        soup=BeautifulSoup(home_page_response.text,'lxml')
        return soup
    
    def get_payload_var(self,soup):
        __VIEWSTATE = soup.find('input',attrs={'id':'__VIEWSTATE'})["value"]
        __VIEWSTATEGENERATOR = soup.find('input',attrs={'id':'__VIEWSTATEGENERATOR'})['value']
        return __VIEWSTATE,__VIEWSTATEGENERATOR
    
    def fetch_data(self,__VIEWSTATE,__VIEWSTATEGENERATOR,icai_number):
        url_link="https://eicmai.in/external/Transaction/OfficesAndFirm/PartnershipFirmList.aspx"
        headers={
            "origin":"https://eicmai.in",
            "referer": "https://eicmai.in/external/Transaction/OfficesAndFirm/PartnershipFirmList.aspx",
            "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
            "content-type": "application/x-www-form-urlencoded; charset=UTF-8"
         }
        payload={
             "ctl00$ContentPlaceHolder1$ScriptManager": "ctl00$ContentPlaceHolder1$updpanel|ctl00$ContentPlaceHolder1$ButtonSearch",
            "__VIEWSTATE":__VIEWSTATE,
            "__VIEWSTATEGENERATOR":__VIEWSTATEGENERATOR,
            "ctl00$ContentPlaceHolder1$DropDownList_Fields": "FirmNo",
            "ctl00$ContentPlaceHolder1$DropDownList_CompOperator":"=",
            "ctl00$ContentPlaceHolder1$TextBox_Value":icai_number,
            "__EVENTTARGET": "",
            "__EVENTARGUMENT":"",
            "__ASYNCPOST": "true",
            "ctl00$ContentPlaceHolder1$ButtonSearch": "Search",
        }
        response=session.post(self.url_link,headers=headers,data=payload)
        return response
    
    def extract_data(self,soup):
        
        table_data = soup.find("table",class_ = "secondline")
        
        headers = [th.text.strip() for th in table_data.find_all('th')]
        rows = table_data.find_all('tr')
        data = []
        
        for row in rows:
            cells = row.find_all('td')
            if cells:
                items = {headers[i]: cells[i].text.strip() for i in range(len(headers))}
                data.append(items)
        return json.dumps(data, indent=4)
   
    def get_output(self):
        for retry_num in range(1):
            self.session = requests.session()
            
            home_page_response = self.home_page()
            
            parsed_data = self.parse_with_lxml(home_page_response)
            
            VIEWSTATE,VIEWSTATEGENERATOR = self.get_payload_var(parsed_data)
            
            second_response = self.fetch_data(VIEWSTATE,VIEWSTATEGENERATOR,self.icai_number)
            
            second_soup = self.parse_with_lxml(second_response)
            
            output = self.extract_data(second_soup)
            return output

In [7]:
if __name__ == '__main__':
    print(PartnershipFirmList("000023").get_output())

[
    {
        "SlNo.": "1",
        "Firm No": "000023",
        "Firm Name": "N I Mehta & Co.",
        "Firm Type": "Partnership",
        "Constitution date": "21/06/1989",
        "Deed date": "03/05/2021",
        "Region": "WESTERN",
        "Country": "INDIA",
        "State": "MAHARASHTRA",
        "City": "MUMBAI",
        "View": ""
    }
]
